# Preparación de blanco terapeutico/traget

A partir de este punto, la estructura obtenida de PDB será nombrada como "blanco terapéutico". Debido a los requerimientos de AutodockVina, es necesario hacer la conversión del archivo .PDB a .PDBQT Además, se realizarán correcciones respecto a cargas e hidrógenos en la estructura.

In [5]:
#Seleccion de atomos 
from pathlib import Path
import MDAnalysis as mda
u = mda.Universe(pdb_path)
print(u)
protein = u.select_atoms("protein")
ligand = u.select_atoms("resname 8PR") #el nombre del ligando va en funcion del ligando unido a la proteina seleccionada
water = u.select_atoms("resname HOH") #solo se llena en caso de que se quieran seleccionar las moleculas de agua
water 
 #Generacion de nuevo archivo .pdb
protein_pdb_path = f"{protein_directory}/protein_{pdb_id}.pdb"
protein.write(protein_pdb_path)
protein_pqr_path = f"{protein_directory}/protein_{pdb_id}.pqr"

# Comando para ejecutar PDB2PQR
import subprocess
pdb2pqr_command = [
    "pdb2pqr",
    "--ff=AMBER",  # Usar el campo de fuerza AMBER
    "--with-ph=7.0",  # Configurar pH
    protein_pdb_path,  # Archivo de entrada
    protein_pqr_path  # Archivo de salida
]

try:
    subprocess.run(pdb2pqr_command, check=True)
    print(f"Archivo PQR generado en: {protein_pqr_path}")
except subprocess.CalledProcessError as e:
    print(f"Error al ejecutar PDB2PQR: {e}")
    
pdbqt_directory = "pdbqt_structures" #directorio definido para los archivos pdbqt
os.makedirs(pdbqt_directory, exist_ok=True)  # Crea el directorio si no existe

# Carga la estructura PQR
pqr_path = f"{protein_directory}/protein_{pdb_id}.pqr"
u = mda.Universe(pqr_path)

pdbqt_path = f"{pdbqt_directory}/{pdb_id}.pdbqt" #conversion a PDBQT
u.atoms.write(pdbqt_path)

print(f"Archivo PDBQT generado y guardado en: {pdbqt_path}")

with open(f"{pdbqt_directory}/{pdb_id}.pdbqt", 'r') as file:
    file_content = file.read()
#MDAnalysis genera dos lineas que autodockVina no reconoce, por lo que es importante modificarlas 

# Reemplazar 'TITLE' y 'CRYST1' con 'REMARK'
file_content = file_content.replace('TITLE', 'REMARK').replace('CRYST1', 'REMARK')
with open(f"{pdbqt_directory}/{pdb_id}.pdbqt", 'w') as file:
    file.write(file_content)

NameError: name 'pdb_path' is not defined